## Time Series Analysis

- **Exponential Smoothing**
- **Autoregressive Models**
- **Linear Regression**

### 1. Import Libraries

#### Install any missing libraries
```python 
!pip install scikit-learn statsmodels pmdarima 
```

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import patsy as pt
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

### 2. Import Data

```python 
df = pd.read_excel(file) # if excel file
df = pd.read_csv(file)   # if csv file
```

Use **pd.to_datetime('col')** to specify the date variable if Date is available in the da
- If there is year and quarter: df['Date'] = pd.PeriodIndex(year=df['Year'], quarter=df['Quarter'], freq='Q').to_timestamp()
- If only year: df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-01-01')
- If year and month: df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01')

### 3. Convert the Data Frame into a  Time Series

```python 
ts = df.set_index('Date')['col'].asfreq('MS')
ts = ts.interpolate(limit_direction='both') # To fill in missing values
```
asfreq - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.asfreq.html

### 4. Divide the data into training and testing sets

```python 
size = 20
train = ts.iloc[:-size]
test = ts.iloc[-size:]
```


### 5. Visualize the Data to see any meaningful patterns

```python 
plt.plot(ts.index, ts.values, label='----')
plt.show()
# add title, legend, customize axes and other options as needed

decomp = seasonal_decompose(train)
fig = decomp.plot()
```
seasonal_decompose - https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html

### 6. Develop the models - Exponential Smoothing

**Exponential Smoothing**
$$
\begin{aligned}
\ell_t &= \alpha y_t + (1 - \alpha)\ell_{t-1} \\
\hat{y}_{t+1|t} &= \ell_t
\end{aligned}
$$

- $\ell_t$: level (smoothed value)  
- $\alpha \in (0,1)$: smoothing constant  
- $\hat{y}_{t+1|t}$: one-step forecast  
  
```python 
ses_model = ExponentialSmoothing(train, trend=None, seasonal=None).fit(smoothing_level=0.8, optimized=True)
ses_forecast = pd.Series(ses_model.forecast(size), index=test.index, name='SES')
```



**Holt's Linear Method**
$$
\begin{aligned}
\ell_t &= \alpha y_t + (1 - \alpha)(\ell_{t-1} + b_{t-1}) \\
b_t &= \beta (\ell_t - \ell_{t-1}) + (1 - \beta)b_{t-1} \\
\hat{y}_{t+h|t} &= \ell_t + h b_t
\end{aligned}
$$

- $\ell_t$: level  
- $b_t$: trend  
- $\alpha, \beta \in (0,1)$: smoothing parameters

   
```python 
holt_model = ExponentialSmoothing(train, trend='add', seasonal=None).fit(smoothing_level=0.8,smoothing_trend=0.2, optimized=True)
holt_forecast = pd.Series(ses_model.forecast(size), index=test.index, name='Holt')
```



**Winter Holt's Method - Trend + Seasonality**
$$
\begin{aligned}
\ell_t &= \alpha (y_t - s_{t-m}) + (1 - \alpha)(\ell_{t-1} + b_{t-1}) \\
b_t &= \beta (\ell_t - \ell_{t-1}) + (1 - \beta)b_{t-1} \\
s_t &= \gamma (y_t - \ell_t) + (1 - \gamma)s_{t-m} \\
\hat{y}_{t+h|t} &= \ell_t + h b_t + s_{t+h-m\lfloor (h-1)/m \rfloor}
\end{aligned}
$$

- $m$: seasonal period (4 for quarters)  
- $\alpha, \beta, \gamma$: smoothing constants  

```python
hw_model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=4).fit(smoothing_level=0.8, smoothing_trend=0.2, smoothing_seasonal = 0.2, optimized=True)
hw_forecast = pd.Series(hw_model.forecast(size), index=test.index, name = 'Winter Holt')
```

**ARIMA**

| Component                         | Parameter | What It Does                                           | Scope                 |
| --------------------------------- | --------- | ------------------------------------------------------ | --------------------- |
| **AR (AutoRegressive)**           | `p` | Uses past values                                       | Short-term / Seasonal |
| **I (Integrated / Differencing)** | `d` | Removes trends or seasonal drift                       | Short-term / Seasonal |
| **MA (Moving Average)**           | `q` | Uses past forecast errors                              | Short-term / Seasonal |
| **Seasonal period**               | `m` | Defines how many steps per season (e.g. 4 = quarterly) | Seasonal              |

```python
arima_model = SARIMAX(
    train,
    order=(1,1,1),
    seasonal_order=(1,1,1,4),
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)

arima_forecast = pd.Series(arima_model.forecast(steps=size), index=test.index,name = 'SARIMAX')
```

**Regression for Time Series**

$$
y_t = \beta_0 + \beta_1 t + \sum_{q=2}^{4} \gamma_q \cdot I\{\text{Quarter}_t = q\} + \varepsilon_t
$$

- $\displaystyle y_t$: observed value at time $t$  
- $t$: time index (trend term)  
- $I\{\text{Quarter}_t = q\}$: indicator variable (dummy) for quarter $q$  
- $\beta_0, \beta_1, \gamma_q$: regression coefficients  
- $\varepsilon_t$: random error term


```python
# Trend (1, 2, 3, ...)
t_train = np.arange(1, len(train) + 1, dtype=float)
t_test  = np.arange(len(train) + 1, len(train) + size + 1, dtype=float)

# Season as integers 1..12 using arange (wrap every 12)
season_train = ((t_train - 1) % 12) + 1
season_test  = ((t_test  - 1) % 12) + 1

# Design matrices: const, trend, numeric season
X_train = pd.DataFrame({'const': 1.0, 't': t_train, 'season': season_train}, index=train.index).astype(float)
X_test = pd.DataFrame({'const': 1.0, 't': t_test, 'season': season_test}, index=test.index).astype(float)

y_train = train.to_numpy(dtype=float)

# Fit and predict
reg = sm.OLS(y_train, X_train).fit()
pred = pd.Series(reg.predict(X_test), index=test.index, name='Regression')
reg.summary()

```

### 6. Visualize the results

```python
plt.figure(figsize=(10,4))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(arima_forecast.index, arima_forecast, label='ARIMA(2,1,2)')
plt.legend()
plt.show()
```

### 7. Evaluate the errors

```python
mse = mean_squared_error(actual, predicted)
mae = mean_absolute_error(actual, predicted)
rmse = np.sqrt(mse)
```